In [22]:
def mean_vector(X):
    rows, cols = len(X), len(X[0])
    mean = [0] * cols
    for i in range(rows):
        for j in range(cols):
            mean[j] += X[i][j]
    return [m / rows for m in mean]


In [24]:
def transpose(matrix):
    rows, cols = len(matrix), len(matrix[0])
    return [[matrix[j][i] for j in range(rows)] for i in range(cols)]

def matrix_multiply(A, B):
    rows_A, cols_A = len(A), len(A[0])
    rows_B, cols_B = len(B), len(B[0])
    result = [[0] * cols_B for _ in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]
    return result


In [26]:
def compute_scatter_matrices(X, y):
    classes = set(y)
    n_features = len(X[0])
    S_W = [[1e-6 if i == j else 0 for j in range(n_features)] for i in range(n_features)]
    S_B = [[0] * n_features for _ in range(n_features)]
    overall_mean = mean_vector(X)
    
    for cls in classes:
        X_class = [X[i] for i in range(len(y)) if y[i] == cls]
        mean_vec = mean_vector(X_class)
        for row in X_class:
            diff = [[row[i] - mean_vec[i]] for i in range(n_features)]
            scatter_temp = matrix_multiply(diff, transpose(diff))
            S_W = [[S_W[i][j] + scatter_temp[i][j] for j in range(n_features)] for i in range(n_features)]
        
        mean_diff = [[mean_vec[i] - overall_mean[i]] for i in range(n_features)]
        scatter_B = matrix_multiply(mean_diff, transpose(mean_diff))
        S_B = [[S_B[i][j] + len(X_class) * scatter_B[i][j] for j in range(n_features)] for i in range(n_features)]
    
    return S_W, S_B

In [28]:
def inverse_matrix(matrix):
    size = len(matrix)
    identity = [[1 if i == j else 0 for j in range(size)] for i in range(size)]
    for i in range(size):
        if matrix[i][i] == 0:
            for k in range(i + 1, size):
                if matrix[k][i] != 0:
                    matrix[i], matrix[k] = matrix[k], matrix[i]
                    identity[i], identity[k] = identity[k], identity[i]
                    break
        factor = matrix[i][i]
        for j in range(size):
            matrix[i][j] /= factor
            identity[i][j] /= factor
        for k in range(size):
            if k != i:
                factor = matrix[k][i]
                for j in range(size):
                    matrix[k][j] -= factor * matrix[i][j]
                    identity[k][j] -= factor * identity[i][j]
    return identity

In [30]:
def lda_fit(X, y, n_components):
    S_W, S_B = compute_scatter_matrices(X, y)
    S_W_inv = inverse_matrix(S_W)
    eig_matrix = matrix_multiply(S_W_inv, S_B)
    eigvecs = transpose(eig_matrix)[:n_components]
    return transpose(eigvecs)

def lda_transform(X, W):
    return matrix_multiply(X, W)

# Example usage
if __name__ == "__main__":
    X = [[2, 3], [3, 4], [4, 5], [6, 8], [7, 9], [8, 10]]
    y = [0, 0, 0, 1, 1, 1]
    W = lda_fit(X, y, n_components=1)
    X_lda = lda_transform(X, W)
    print("Transformed data:", X_lda)

Transformed data: [[3000016.8746119887], [3000023.624611145], [3000030.374610301], [6000047.24922229], [6000053.999221448], [6000060.749220602]]
